# Power BI Management Module Authentication - Service Principal

**Author**: Melissa Coates  

**Last updated and tested**: August 22, 2022 

**Purpose:** This notebook is a guided learning experience to get familiar with how to authenticate to the Power BI Management Module using a service principal.

**When to use a service principal:**

- When executing scheduled, unattended, operations.
- When you have been granted authority to to run Power BI admin APIs, but you personally aren't a Power BI administrator.
- When there's a technical blocker to using your user account to run Power BI admin APIs (ex: MFA is enabled, password hash isn't synchronized).

**Prerequisites**:

The [<u>service principal</u>](https://docs.microsoft.com/en-us/power-bi/enterprise/read-only-apis-service-principal-authentication) requires:

1. An Azure AD app to be registered (and its associated service principal).
2. A secret created for the app. (Tip: watch out for expiration dates - it defaults to 1 year.) 
3. Azure AD app membership in an Azure AD group. (Tip: suggested group name: Power BI Service Principals).
4. Power BI tenant setting "Allow service principals to use read-only Power BI admin APIs" is enabled for your Azure AD group (from #3).

**Tips**:

- Do NOT set up API permissions in Azure AD for your Azure AD app; in fact that causes an unauthorized error. Use of the Azure AD group & the tenant setting per (3) and (4) above are what controls tenant-wide read permissions for Power BI. Plus, workspace permissions where relevant. [<u>More info</u>](https://docs.microsoft.com/en-us/power-bi/enterprise/read-only-apis-service-principal-authentication).

## **Login to the Power BI Service Using a Service Principal**

* * *

**Update in the script before executing:**

1. **Line 1: Azure AD app ID** (aka client ID)
2. **Line 2: Azure AD tenant** ID (aka directory ID)

**The script below will interactively prompt you for:**

1. **Azure AD app secret** (aka client secret / app password)

**Comments:**

- Because this is for learning purposes, this script interactively prompts for the service principal secret to be passed into a credential object. 
- When you operationalize any of your scripts:
    - The prompt for the app secret will need to be removed. If you're using a VM or similar, consider using the [Secret Management Module](https://www.powershellgallery.com/packages/Microsoft.PowerShell.SecretManagement) published by Microsoft for managing the secret value securely in either a local vault or in Azure Key Vault. Do NOT store secrets, keys, or passwords directly in any of the scripts (or in a separate file if it's unencrypted or trivial to decrypt). Alternatively, you could use Azure Key Vault with services like Azure Functions, Azure Automation, or Azure Data Factory.
    - You might also store the app ID and tenant ID securely as well, once you have a solution for handling the secret.

In [ ]:
[string]$AzureADAppID = '1234-1234'
[string]$AzureTenantID = '1234-1234'

#Prompt for Azure AD App secret (so that it isn't saved with the script): 
[securestring]$AzureADAppSecret = Read-Host -Prompt "Input secret for $AzureADAppID" -AsSecureString

#Create temporary credential object: 
[pscredential]$CredentialObj = New-Object System.Management.Automation.PSCredential($AzureADAppID, $AzureADAppSecret)  

#Log into Power BI Service with credential:
Connect-PowerBIServiceAccount -Credential $CredentialObj -ServicePrincipal -Tenant $AzureTenantID

## **Test to Verify Data is Returned: Organization Scope**  
---
## <span style="font-size: 14px;">This script will check that data for the tenant is able to be returned. This script:</span>  

- Uses the [<u>Get-PowerBIWorkspace</u>](https://docs.microsoft.com/en-us/powershell/module/microsoftpowerbimgmt.workspaces/get-powerbiworkspace) cmdlet from the Power BI Management Module.
- Uses the "Organization" scope, which means that Power BI administrator permissions are required (or Power Platform administrator or global administrator). Metadata can be returned whether or not the administrator possesses workspace permissions.
- Will return all active workspaces. Deleted and orphaned workspaces will be omitted.
- Will display 3 columns, formatted in a table.
- If -All is not specified, by default this cmdlet returns the first 100 workspaces.

Note that the "Organization" scope is able to display a list of personal workspaces, in addition to standard workspaces.

Tip: A service principal can also return data based on user scope, if the service principal has been granted workspace permissions. That's possible, just less common.

In [ ]:
Get-PowerBIWorkspace -Scope Organization | Where-Object State -eq 'Active' | Format-Table Name, Type, IsOnDedicatedCapacity